In [1]:
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QToolTip, \
     QPushButton,QMessageBox,QDesktopWidget, QLabel
from PyQt5.QtGui import QFont,QIcon,QPixmap,QImage
from PyQt5.QtCore import QTimer
import cv2
import picture as pic
from skimage import transform
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
import time
from PIL import Image



show_ret={}

class myWindow(QWidget):
    def __init__(self,parent = None):
        super(myWindow,self).__init__(parent)
 
        self.timer_camera = QTimer()
        self.cap = cv2.VideoCapture()
        self.initUI()
        self.slot_init()
 
 
    def initUI(self):
 
        self.mylabel()
        self.myButton()
        self.myLabelPic()
 
        self.setFixedSize(670,520)
        self.center()
        self.setWindowIcon(QIcon('icon.jpg'))
        self.setWindowTitle('gesture recognition')
 
 
    def mylabel(self):
 
        label_roi = QLabel('原图',self)
        label_roi.setStyleSheet("QLabel{font-size:18px;}")
        label_roi.resize(60,30)
        label_roi.move(120,15)
 
        label_res = QLabel('轮廓线', self)
        label_res.setStyleSheet("QLabel{font-size:18px;}")
        label_res.resize(60, 30)
        label_res.move(480, 15)
 
        label_pre = QLabel('预测', self)
        label_pre.setStyleSheet("QLabel{font-size:20px;}")
        label_pre.resize(50,30)
        label_pre.move(400,400)
 
        label_result = QLabel('结果', self)
        label_result.setStyleSheet("QLabel{font-size:20px;}")
        label_result.resize(50, 30)
        label_result.move(400,430)
 
    def myLabelPic(self):
        self.label_show_roi = QLabel(self)
        self.label_show_roi.setFixedSize(301,301)
        self.label_show_roi.move(20,50)
        self.label_show_roi.setStyleSheet("QLabel{background:white;}")
        self.label_show_roi.setAutoFillBackground(True)
 
        self.label_show_ret = QLabel(self)
        self.label_show_ret.setFixedSize(301, 301)
        self.label_show_ret.move(350, 50)
        self.label_show_ret.setStyleSheet("QLabel{background:white;}")
        self.label_show_ret.setAutoFillBackground(True)
 
        self.label_show_recognition = QLabel('0',self)
        self.label_show_recognition.setStyleSheet("QLabel{background:white;}")
        self.label_show_recognition.setStyleSheet("QLabel{font-size:50px;}")
        self.label_show_recognition.setFixedSize(100,100)
        self.label_show_recognition.move(500, 380)
        self.label_show_recognition.setAutoFillBackground(True)
 
    def myButton(self):
        QToolTip.setFont(QFont('SansSerif', 10))
 
        self.button_open_camera = QPushButton('打开相机', self)
        self.button_open_camera.setToolTip('按i,k,j,l可以进行上下左右调整')
        self.button_open_camera.resize(100,30)
        self.button_open_camera.move(100, 400)
 
        self.butoon_recognition = QPushButton('进行预测', self)
        self.butoon_recognition.setFixedSize(100, 30)
        self.butoon_recognition.move(100, 450)
 
 
    def slot_init(self):
        self.button_open_camera.clicked.connect(self.button_open_camera_click)
        self.butoon_recognition.clicked.connect(self.button_recognition_click)
        self.timer_camera.timeout.connect(self.show_camera)
 
    def button_open_camera_click(self):
        if self.timer_camera.isActive() == False:
            self.cap.open(0)
            self.timer_camera.start(30)
            self.button_open_camera.setText(u'关闭相机')
        else:
            self.timer_camera.stop()
            self.cap.release()
            self.label_show_roi.clear()
            self.label_show_ret.clear()
            self.label_show_recognition.setText('0')
            self.button_open_camera.setText(u'打开相机')
 
    
    def show_camera(self):
        width, height = 300, 300  # 设置拍摄窗口大小
        x0, y0 = 100, 100  # 设置选取位置
        flag, frame = self.cap.read()
        global ret
        roi, res, ret = pic.binaryMask(frame, x0, y0, width, height)
        #roi, res, ret, self.fourier_result, self.efd_result = pic.binaryMask(frame, x0, y0, width, height)
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        show_roi = QImage(roi.data, roi.shape[1], roi.shape[0], QImage.Format_RGB888)
        show_ret = QImage(ret.data, ret.shape[1], ret.shape[0], QImage.Format_Grayscale8)
        self.label_show_roi.setPixmap(QPixmap.fromImage(show_roi))
        self.label_show_ret.setPixmap(QPixmap.fromImage(show_ret))

    def button_recognition_click(self):   
        
        im2=Image.fromarray(np.uint8(ret))
        im2.save('flag.jpg')
        img=cv2.imread('flag.jpg')
        img=transform.resize(img, (150, 150))
        img=img.reshape(1,150,150,3)
        predict = model.predict_classes(img)
        if predict==0:
            RR="布"
        elif(predict==1):
            RR="石头"
        elif(predict==2):
            RR="剪刀"
        else:
            RR="其他"
        self.label_show_recognition.setText(str(RR))  
        cv2.waitKey(0)
 
    def closeEvent(self, QCloseEvent):
        reply = QMessageBox.question(self, 'Message',"Are you sure to quit?",
                                     QMessageBox.Yes |QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            if self.cap.isOpened():
                self.cap.release()
            if self.timer_camera.isActive():
                self.timer_camera.stop()
            QCloseEvent.accept()
        else:
            QCloseEvent.ignore()
 
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
 
if __name__ == "__main__":
    print("loading")
    model = load_model('rps16.h5')  #选取自己的.h模型名称
    app = QApplication(sys.argv)
    win = myWindow()
    win.show()
    sys.exit(app.exec())

e:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
e:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
e:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
e:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

loading
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


SystemExit: 0

e:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
